# RAG

RAG, ou Retrieval-Augmented Generation (Geração Aumentada por Recuperação), é uma técnica híbrida que combina modelos de recuperação de informações com modelos de geração de texto (geralmente LLMs, como GPT ou BERT). O objetivo do RAG é melhorar a capacidade de um modelo de linguagem em fornecer respostas mais precisas e informadas, especialmente em cenários onde o modelo pode não ter todos os dados contextuais na memória ou no treinamento.

Como o RAG funciona:
1. Recuperação de Informação (Retrieval): 

* Em vez de confiar exclusivamente no conhecimento armazenado no modelo de linguagem, o sistema RAG inclui uma etapa de recuperação de informações. O modelo de recuperação (como um mecanismo de busca ou uma loja de vetores) busca informações relevantes em uma base de dados externa, como documentos, artigos ou outros textos, que são relacionados à consulta do usuário. 
* Isso é útil quando o modelo de linguagem não tem conhecimento completo ou atualizado sobre o assunto, permitindo que ele acesse novas informações ou dados que não estavam disponíveis durante o treinamento.

2. Geração de Resposta (Generation):

* Após a recuperação das informações relevantes, essas informações são passadas para um modelo de geração de texto. O modelo de linguagem (como GPT ou BERT) então processa a consulta original do usuário e as informações recuperadas para gerar uma resposta mais precisa e detalhada.
* O modelo não apenas repete o que foi recuperado, mas integra esses dados com sua própria capacidade de gerar linguagem, criando uma resposta fluida, coerente e contextual.

## Chatbot com dados próprios

Importante: Executar o pop de pysqlite3 apenas se estiver em MAC OS ou Linux

In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

Vamos começar a codificar, primeiro importando as blibliotecas para criar o CHAT.Vamos usar um modelo opensource Llama-3.2-3B-Instruct, para ter acesso a ele, precisa ter uma conta no Hugging Face e solicitar acesso ao modelo em: https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

In [2]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint

model = 'meta-llama/Llama-3.2-3B-Instruct' #Vamos usar um modelo opensource.

llm = HuggingFaceEndpoint(repo_id=model) # na primeira execução, o modelo será baixado
chat = ChatHuggingFace(llm=llm) 

/home/dimmy/Projects/dimmy/chat_langchain/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/dimmy/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Criando estrutura de Retrieval

1. Chroma de langchain_community.vectorstores.chroma: O Chroma é uma implementação de uma loja de vetores (vector store), que é usada para armazenar e recuperar representações vetoriais de textos ou documentos. Essas representações são úteis em tarefas de busca semântica e recuperação de informações, permitindo consultas mais eficientes e relevantes. O Chroma é uma opção popular de loja de vetores na comunidade de LangChain, usada em combinação com embeddings para indexar e buscar dados de texto de forma eficaz.
PyPDFLoader de langchain_community.document_loaders.pdf:

2. O PyPDFLoader é um carregador de documentos específico para arquivos PDF. Ele utiliza a biblioteca PyPDF para ler e carregar o conteúdo dos PDFs em um formato que possa ser usado dentro da aplicação LangChain, transformando o texto em algo que possa ser processado posteriormente, como para extração de informações ou divisão do texto em segmentos menores.
RecursiveCharacterTextSplitter de langchain_text_splitters:

3. O RecursiveCharacterTextSplitter é uma ferramenta que divide um texto em partes menores. Essa técnica é útil para garantir que os segmentos de texto estejam dentro dos limites de tokens que podem ser processados por um LLM. O algoritmo recursivo tenta dividir o texto em quebras naturais, como parágrafos ou sentenças, para manter o contexto dentro das partes, mesmo com grandes documentos.

In [3]:
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
# listando arquivos na pasta files
import os
dir = 'files'
files = os.listdir(dir)
print(files)

['AUTISMO - PRATICAS BASEADAS EM EVIDENCIAS.pdf', 'TranstornoEspectroAutista.pdf']


Carregando arquivos em pages usando PyPDFLoader

In [5]:
pages = []
for path in files:
    loader = PyPDFLoader(os.path.join(dir, path))
    pages.extend(loader.load())

In [6]:
print(len(pages))

517


As páginas podem conter conteúdo demais, ou seja, com janelas de contexto que os modelos de LLM não conseguem armazenar. Para tanto precisamos quebrar esses textos em elementos semanticamente válidos. Para isso vamos usar o RecursiveCharacterTexetSplitter

In [7]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000, #tamanho do chunk (pedaço) do texto
    chunk_overlap=100, #overplap entre os chunks, ou seja, quantos caracteres de um chunk são repetidos no próximo (10%)
    separators=["\n\n", "\n", ".", " ", ""]
)

#Executando o split dos documentos
documents = recur_split.split_documents(pages)
print("Quantidade de páginas: ", len(documents))

Quantidade de páginas:  1527


In [8]:
documents[0]

Document(page_content='Autismo: compreensão e práticas baseadas em evidências\nAUTISMO:\nCOMPREENSÃO E PRÁTICAS\nBASEADAS EM EVIDÊNCIAS', metadata={'source': 'files/AUTISMO - PRATICAS BASEADAS EM EVIDENCIAS.pdf', 'page': 2})

In [9]:
#Modificando os metadados dos documentos
for i, doc in enumerate(documents, 1):
    doc.metadata['source'] = doc.metadata['source'].replace('files/', '')
    #Aqui podemos colocar algo como keywords, autores, etc..
    doc.metadata['keywords'] = "Autismo, Educação, Inclusão"
    doc.metadata['doc_id'] = i

Depois dos dados seperadados vamos armazenar em um vector_store. Um vector store é uma estrutura de armazenamento que guarda representações vetoriais (embeddings) de dados, geralmente texto, para facilitar a recuperação de informações ou a busca semântica. É uma peça fundamental em sistemas de inteligência artificial que trabalham com modelos de linguagem natural (LLMs), principalmente em tarefas de busca, recomendação e análise de similaridade.

Como precisamos de um embedding de cada texto, precisamos de uma forma de transformar o texto em embedding. Iremos usar também de código aberto. Solicite acesso a ele em: https://huggingface.co/meta-llama/Meta-Llama-3-8B

In [10]:
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings

model = 'thuan9889/llama_embedding_model_v1'
embedding_model = HuggingFaceBgeEmbeddings(model_name = model) # na primeira execução, o modelo será baixado

In [11]:
dir_vector_store = 'vector_stores/chat_retrieval_db_omni'

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=dir_vector_store
)

## Criando cadeias de execuções 

In [12]:
# Carregando vector store (caso tenha sido salvo):
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=dir_vector_store
)

In [13]:
from langchain.chains.retrieval_qa.base import RetrievalQA

Para deixar o exemplo mais completo, usaremos um prompt_template a fim de personalizar nosso agente

In [14]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
    """
    Você é um agente inteligente chamada Rebeca, que trabalha em uma organização de apoio a pessoas com autismo.
    Utilize o contexto fornecido para responder à pergunta com empatia e precisão, considerando que o tema é autismo.
    Se você não souber a resposta com certeza, informe que não sabe e não tente especular.
    Mantenha a resposta breve e clara, em no máximo três frases, e sempre priorize um tom acolhedor e respeitoso.

    Contexto: {context}

    Pergunta: {question}

    Resposta:
    """)

In [15]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'), # max marginal relevance
                                                        # que é um algoritmo de diversidade de documentos
    chain_type_kwargs={'prompt': chain_prompt}, #adicionando o template
    return_source_documents=True #quais documentos foram usados para responder
)

In [16]:
question = 'autismo só afeta crianças?'
response = chat_chain.invoke({'query': question})

In [17]:
response['result']

'Não, autismo não só afeta crianças, embora a maioria dos casos seja diagnosticada em crianças e adolescentes. Alguns adultos também podem ser diagnosticados com autismo, e muitas vezes, os sintomas podem não serem evidentes até a idade adulta.'

In [18]:
response['source_documents'][0].metadata

{'page': 59, 'source': 'files/AUTISMO - PRATICAS BASEADAS EM EVIDENCIAS.pdf'}

In [19]:
from langchain.globals import set_debug

set_debug(True)

question = 'Como o autismo se manifesta?'
response = chat_chain.invoke({'query': question})

set_debug(False)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Como o autismo se manifesta?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Como o autismo se manifesta?",
  "context": "Autismo: compreensão e práticas baseadas em evidências\n\nAutismo: compreensão e práticas baseadas em evidências\n\nAutismo: compreensão e práticas baseadas em evidências\n\nAutismo: compreensão e práticas baseadas em evidências"
}
[llm/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain > llm:ChatHuggingFace] Entering LLM run with input:
{
  "prompts": [
    "Human: \n    Você é um agente inteligente chamada Rebeca, que trabalha em uma organização de apoio a pessoas com autismo.\n    Utilize o contexto fornecido para responder à pergunta com empatia e precisão, considerando que o tema é a

In [20]:
response

{'query': 'Como o autismo se manifesta?',
 'result': 'Olá! Entendo que você está procurando entender como o autismo se manifesta. O autismo, também conhecido como transtorno do espectro autista (TEA), é uma condição neurológica que afeta a comunicação, interação social e comportamento. As pessoas com autismo podem apresentar uma ampla gama de habilidades e desafios, e a expressão da condição pode variar significativamente de uma pessoa para outra.',
 'source_documents': [Document(page_content='Autismo: compreensão e práticas baseadas em evidências', metadata={'page': 11, 'source': 'files/AUTISMO - PRATICAS BASEADAS EM EVIDENCIAS.pdf'}),
  Document(page_content='Autismo: compreensão e práticas baseadas em evidências', metadata={'page': 13, 'source': 'files/AUTISMO - PRATICAS BASEADAS EM EVIDENCIAS.pdf'}),
  Document(page_content='Autismo: compreensão e práticas baseadas em evidências', metadata={'page': 3, 'source': 'files/AUTISMO - PRATICAS BASEADAS EM EVIDENCIAS.pdf'}),
  Document(pag

## Criando o loop de chat

In [21]:
while True:
    question = input('Pergunta: ')
    if question.upper() in ['SAIR', "Q"]:
        break
    print("Pergunta: ", question)
    try:
        response = chat_chain.invoke({'query': question})
        print("Resposta:", response['result'])
    except Exception as e:
        print("Resposta: Não entendi a pergunta. Pode reformular?")

Resposta: Meu nome é Rebeca, e estou aqui para ajudar e apoiar as pessoas com autismo e suas famílias. Estou comprometida em fornecer respostas informadas e respeitosas, com base em conhecimentos atualizados e em consulta com especialistas no campo do autismo.
Resposta: Meu nome é Rebeca, e estou aqui para ajudar e apoiar as pessoas com autismo e suas famílias. Estou comprometida em fornecer respostas informadas e respeitosas, com base em conhecimentos atualizados e em consulta com especialistas no campo do autismo.
Resposta: Olá! Eu sou Rebeca, e estou aqui para ajudar. O autismo é uma condição neurológica que afeta a comunicação, o comportamento e o desenvolvimento social. É caracterizado por alterações significativas em como as pessoas processam e interpretam informações sensoriais, o que pode afetar sua capacidade de interagir com o mundo ao seu redor.
Resposta: Olá! Eu sou Rebeca, feliz em ajudar.

A partir do contexto fornecido, posso dizer que a pergunta parece estar relacionada